### 0. Helper functions

In [21]:
import numpy as np

class bin_data:
    def __init__(self, env):
        self.total_capacity = np.full(env.num_bins, 0) # Total capacity seen across all trials
        self.capacity_used = np.full(env.num_bins, 0) # Total capacity used across all trials
        self.steps_taken = 0 # Number of timesteps taken before trial terminated
        self.placed = 0 # Number of items correctly placed
        self.misplaced = 0
        self.discarded = 0
        self.data_points = 0 # Number of trials we've witnessed
    

    def log(self, env):
        self.total_capacity = np.add(self.total_capacity, env.capacity)
        self.capacity_used = np.add(self.capacity_used, env.state[:-1])
        self.steps_taken += sum(env.logs.values())
        self.placed += env.logs["placed"]
        self.misplaced += env.logs["misplaced"]
        self.discarded += env.logs["discarded"]
        self.data_points += 1
        

    def get_avg(self):
        percentages = 100 - ((self.capacity_used/self.total_capacity) * 100)
        
        return {
            "steps" : self.steps_taken / self.data_points,
            "utilization" : sum(percentages) / len(percentages),
            "accuracy" : (self.placed / (self.placed + self.misplaced)) * 100
        }
    

    def print_data(self):
        data = self.get_avg()
        print(f'Average number of steps taken: {data["steps"]}')
        print(f'Average bin utilization: {round(data["utilization"], 2)}%')
        print(f'Accuracy: {round(data["accuracy"], 2)}%')

### 1. Import dependencies

In [ ]:
!pip install tensorflow==2.7.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [2]:
from BinPackingEnvironment1D import BinPacking

### 2. Create environment

In [3]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [4]:
env.action_space.sample()

6

In [5]:
env.observation_space.sample()

array([ 1, 19,  4, 19,  5,  1,  9,  6, 12, 15,  8])

### 3. Run baseline test (No ML)

In [6]:
control_data = bin_data(env)

In [20]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    print()
    control_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-3133
{'placed': 47, 'misplaced': 301, 'discarded': 34}

Episode:2 Score:-2625
{'placed': 45, 'misplaced': 250, 'discarded': 34}

Episode:3 Score:-3194
{'placed': 46, 'misplaced': 311, 'discarded': 26}

Episode:4 Score:-2306
{'placed': 44, 'misplaced': 221, 'discarded': 28}

Episode:5 Score:-2119
{'placed': 46, 'misplaced': 206, 'discarded': 21}

Episode:6 Score:-1412
{'placed': 43, 'misplaced': 136, 'discarded': 19}

Episode:7 Score:-4708
{'placed': 47, 'misplaced': 454, 'discarded': 43}

Episode:8 Score:-1867
{'placed': 43, 'misplaced': 179, 'discarded': 24}

Episode:9 Score:-2499
{'placed': 46, 'misplaced': 241, 'discarded': 27}

Episode:10 Score:-3958
{'placed': 47, 'misplaced': 387, 'discarded': 27}



### 4. Train an RL Model

In [8]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [9]:
# Will throw an error if these don't exist
log_path = os.path.join('Training', 'Logs')

In [10]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [11]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=100000)

Logging to Training\Logs\PPO_65
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 445       |
|    ep_rew_mean     | -3.76e+03 |
| time/              |           |
|    fps             | 300       |
|    iterations      | 1         |
|    time_elapsed    | 6         |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 486         |
|    ep_rew_mean          | -4.1e+03    |
| time/                   |             |
|    fps                  | 398         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011210587 |
|    clip_fraction        | 0.0751      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.39       |
|    explained_variance   | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 279         |
|    ep_rew_mean          | -1.94e+03   |
| time/                   |             |
|    fps                  | 598         |
|    iterations           | 11          |
|    time_elapsed         | 37          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.009065133 |
|    clip_fraction        | 0.047       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.98       |
|    explained_variance   | 6.16e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.03e+03    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0113     |
|    value_loss           | 7.27e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 270   

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 186          |
|    ep_rew_mean          | -846         |
| time/                   |              |
|    fps                  | 606          |
|    iterations           | 21           |
|    time_elapsed         | 70           |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0077434834 |
|    clip_fraction        | 0.07         |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.61        |
|    explained_variance   | 6.96e-05     |
|    learning_rate        | 0.0003       |
|    loss                 | 1.15e+03     |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.0135      |
|    value_loss           | 2.4e+03      |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 131         |
|    ep_rew_mean          | -443        |
| time/                   |             |
|    fps                  | 591         |
|    iterations           | 31          |
|    time_elapsed         | 107         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.005614876 |
|    clip_fraction        | 0.0477      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.26       |
|    explained_variance   | 0.569       |
|    learning_rate        | 0.0003      |
|    loss                 | 430         |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.0112     |
|    value_loss           | 1.04e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 129   

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 101        |
|    ep_rew_mean          | -266       |
| time/                   |            |
|    fps                  | 595        |
|    iterations           | 41         |
|    time_elapsed         | 141        |
|    total_timesteps      | 83968      |
| train/                  |            |
|    approx_kl            | 0.00672236 |
|    clip_fraction        | 0.0487     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.22      |
|    explained_variance   | 0.568      |
|    learning_rate        | 0.0003     |
|    loss                 | 324        |
|    n_updates            | 400        |
|    policy_gradient_loss | -0.0126    |
|    value_loss           | 715        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 101         |
|    ep_rew_m

### 5. Save model

In [13]:
#PPO_Path = os.path.join('Training', 'Saved Models', 'Constant_PPO_Model_Discard_Penalty')

In [14]:
#model.save(PPO_Path)

### 6. Load model

In [15]:
#model = PPO.load(PPO_Path, env=env)

### 7. Test model

In [16]:
real_data = bin_data(env)
env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

In [17]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    obs = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    real_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-226
{'placed': 44, 'misplaced': 18, 'discarded': 18}
Episode:2 Score:-265
{'placed': 50, 'misplaced': 3, 'discarded': 57}
Episode:3 Score:-127
{'placed': 43, 'misplaced': 6, 'discarded': 22}
Episode:4 Score:-267
{'placed': 43, 'misplaced': 10, 'discarded': 42}
Episode:5 Score:-269
{'placed': 41, 'misplaced': 5, 'discarded': 52}
Episode:6 Score:-135
{'placed': 45, 'misplaced': 3, 'discarded': 30}
Episode:7 Score:-201
{'placed': 54, 'misplaced': 7, 'discarded': 37}
Episode:8 Score:-76
{'placed': 39, 'misplaced': 2, 'discarded': 19}
Episode:9 Score:-399
{'placed': 46, 'misplaced': 14, 'discarded': 61}
Episode:10 Score:-226
{'placed': 44, 'misplaced': 6, 'discarded': 42}


In [18]:
control_data.print_data()

Average number of steps taken: 462.3
Average bin utilization: 98.5%
Accuracy: 10.24%


In [19]:
real_data.print_data()

Average number of steps taken: 90.3
Average bin utilization: 97.8%
Accuracy: 85.85%
